In [ ]:
### BERT score with human reference
from bert_score import score
import glob
human_path = '/DATA/joosung/controllable_english/evaluation/yelp/human/'
neg_human_DRG = human_path + 'sentiment.test.0.humanDRG'
neg_human_DRG_open = open(neg_human_DRG, "r")
neg_human_DRG_dataset = neg_human_DRG_open.readlines()
neg_human_DRG_open.close()

pos_human_DRG = human_path + 'sentiment.test.1.humanDRG'
pos_human_DRG_open = open(pos_human_DRG, "r")
pos_human_DRG_dataset = pos_human_DRG_open.readlines()
pos_human_DRG_open.close()

neg_human_DUAL = human_path + 'sentiment.test.0.humanDUAL'
neg_human_DUAL_open = open(neg_human_DUAL, "r")
neg_human_DUAL_dataset = neg_human_DUAL_open.readlines()
neg_human_DUAL_open.close()

pos_human_DUAL = human_path + 'sentiment.test.1.humanDUAL'
pos_human_DUAL_open = open(pos_human_DUAL, "r")
pos_human_DUAL_dataset = pos_human_DUAL_open.readlines()
pos_human_DUAL_open.close()

DRG_refs = []
neg_len = len(neg_human_DRG_dataset)
pos_len = len(pos_human_DRG_dataset)
for k in range(neg_len):
    ref_sen = neg_human_DRG_dataset[k].split('\t')[1]
    DRG_refs.append(ref_sen)
for k in range(pos_len):
    ref_sen = pos_human_DRG_dataset[k].split('\t')[1]
    DRG_refs.append(ref_sen)

DUAL_refs = []
neg_len = len(neg_human_DUAL_dataset)
pos_len = len(pos_human_DUAL_dataset)
for k in range(neg_len):
    ref_sen = neg_human_DUAL_dataset[k].split('\t')[1]
    DUAL_refs.append(ref_sen)
for k in range(pos_len):
    ref_sen = pos_human_DUAL_dataset[k].split('\t')[1]
    DUAL_refs.append(ref_sen)
    
neg_out_files = glob.glob(output_path + '*test.0*')
pos_out_files = glob.glob(output_path + '*test.1*')
model_name=[]
score_list=[]
for i in range(len(neg_out_files)):    
    cands = []
    neg_data_open = open(neg_out_files[i], "r")
    neg_data_dataset = neg_data_open.readlines()
    neg_len = len(neg_data_dataset)
    neg_data_open.close()
    
    for k in range(neg_len):
        out_sen = neg_data_dataset[k].split('\t')[1]
        cands.append(out_sen)
    
    pos_out_name = neg_out_files[i].split('.0.')[0]+'.1.'+neg_out_files[i].split('.0.')[-1]
    pos_data_open = open(pos_out_name, "r")
    pos_data_dataset = pos_data_open.readlines()
    pos_len = len(neg_data_dataset)
    pos_data_open.close()
    
    for k in range(pos_len):
        out_sen = pos_data_dataset[k].split('\t')[1]
        cands.append(out_sen)

    P, R, DRG_F1 = score(cands, DRG_refs, lang='en', verbose=True)
    DRG_F1_list=list(DRG_F1.numpy())
    DRG_BERT_scroe = sum(DRG_F1_list)/len(DRG_F1_list)
    
    P, R, DUAL_F1 = score(cands, DUAL_refs, lang='en', verbose=True)
    DUAL_F1_list=list(DUAL_F1.numpy())
    DUAL_BERT_scroe = sum(DUAL_F1_list)/len(DUAL_F1_list)
    
    BERT_score = (DRG_BERT_scroe+DUAL_BERT_scroe)/2    
    
    model_name.append(neg_out_files[i].split('.')[-1])
    score_list.append(BERT_score)
for i in range(len(model_name)):
    print("model: ", model_name[i])
    print('BERT score(F1): {}'.format(score_list[i]*100))
    print('')

In [ ]:
### PPL
import torch, os
from transformers import GPT2Tokenizer, GPT2Model, GPT2LMHeadModel
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device='cpu'
model_class, tokenizer_class = (GPT2LMHeadModel, GPT2Tokenizer)
gpt2_lm_tokenizer = tokenizer_class.from_pretrained('gpt2-large')
gpt2_lm_model = model_class.from_pretrained('gpt2-large')
gpt2_lm_model.to(device)
gpt2_lm_model.eval()

lm_loss = torch.nn.CrossEntropyLoss(ignore_index=-1, reduction='none')

# import logging
# logging.basicConfig(level=logging.INFO)

def calculate_ppl_gpt(sentence_batch):
    # tokenize the sentences
    tokenized_ids = [None for i in range(len(sentence_batch))]
    
    for i in range(len(sentence_batch)):
        tokenized_ids[i] = gpt2_lm_tokenizer.encode(sentence_batch[i])
        
    sen_lengths = [len(x) for x in tokenized_ids]
    max_sen_lenght = max(sen_lengths)
    
    n_batch = len(sentence_batch)
    input_ids = np.zeros( shape=(n_batch, max_sen_lenght), dtype=np.int64)
    lm_labels = np.full(shape=(n_batch, max_sen_lenght), fill_value=-1)
    
    for i, tokens in enumerate(tokenized_ids):
        input_ids[i, :len(tokens)] = tokens
        lm_labels[i, :len(tokens)-1] = tokens[1:] 
    
    input_ids = torch.tensor(input_ids).to(device)
    lm_labels = torch.tensor(lm_labels).to(device)
    with torch.no_grad():
        lm_pred = gpt2_lm_model(input_ids)
    loss_val = lm_loss(lm_pred[0].view(-1, lm_pred[0].size(-1)), lm_labels.view(-1))
    normalized_loss = loss_val.view(n_batch,-1).sum(dim= -1) / torch.tensor(sen_lengths, dtype=torch.float32).to(device)
    #normalized_loss = loss_val.view(n_batch,-1).sum(dim= -1)
    ppl = torch.exp(normalized_loss)
    return  ppl.tolist() 

In [ ]:
import glob
neg_out_files = glob.glob(output_path + '*test.0*')
pos_out_files = glob.glob(output_path + '*test.1*')
for i in range(len(neg_out_files)):
    print("model: ", neg_out_files[i].split('.')[-1])
    
    neg_data_open = open(neg_out_files[i], "r")
    neg_data_dataset = neg_data_open.readlines()
    neg_len = len(neg_data_dataset)
    neg_data_open.close()
    
    neg_PPL = 0
    for k in range(neg_len):
        out_sen = neg_data_dataset[k].split('\t')[1].strip()
        if len(out_sen) != 0:
            sample_PPL = calculate_ppl_gpt([out_sen])[0]
            neg_PPL += sample_PPL
    
    pos_out_name = neg_out_files[i].split('.0.')[0]+'.1.'+neg_out_files[i].split('.0.')[-1]
    pos_data_open = open(pos_out_name, "r")
    pos_data_dataset = pos_data_open.readlines()
    pos_len = len(neg_data_dataset)
    pos_data_open.close()
    
    pos_PPL = 0
    for k in range(pos_len):
        out_sen = pos_data_dataset[k].split('\t')[1].strip()
        if len(out_sen) != 0:
            sample_PPL = calculate_ppl_gpt([out_sen])[0]
            pos_PPL += sample_PPL

    total_PPL = (neg_PPL+pos_PPL)/(neg_len+pos_len)
    print('PPL: {}'.format(total_PPL))

In [1]:
!./e2e-metrics/measure_scores.py /data/private/E2E/dataset/f_test.txt /data/private/E2E/predictions/no_pretrained/try_3/f_pred_1.txt
!./e2e-metrics/measure_scores.py /data/private/E2E/dataset/f_test.txt /data/private/E2E/predictions/no_pretrained/try_3/f_pred_2.txt
!./e2e-metrics/measure_scores.py /data/private/E2E/dataset/f_test.txt /data/private/E2E/predictions/no_pretrained/try_3/f_pred_3.txt
!./e2e-metrics/measure_scores.py /data/private/E2E/dataset/f_test.txt /data/private/E2E/predictions/no_pretrained/try_3/f_pred_4.txt
!./e2e-metrics/measure_scores.py /data/private/E2E/dataset/f_test.txt /data/private/E2E/predictions/no_pretrained/try_3/f_pred_5.txt
!./e2e-metrics/measure_scores.py /data/private/E2E/dataset/f_test.txt /data/private/E2E/predictions/no_pretrained/try_3/f_pred_6.txt
!./e2e-metrics/measure_scores.py /data/private/E2E/dataset/f_test.txt /data/private/E2E/predictions/no_pretrained/try_3/f_pred_7.txt
!./e2e-metrics/measure_scores.py /data/private/E2E/dataset/f_test.txt /data/private/E2E/predictions/no_pretrained/try_3/f_pred_8.txt

/data/private/E2E/dataset/f_test.txt /data/private/E2E/predictions/no_pretrained/try_3/f_pred_1.txt None
Running MS-COCO evaluator...
creating index...
index created!
Loading and preparing results...     
DONE (t=0.00s)
creating index...
index created!
tokenization...
PTBTokenizer tokenized 129948 tokens at 618160.09 tokens per second.
PTBTokenizer tokenized 13259 tokens at 132508.62 tokens per second.
setting up scorers...
computing METEOR score...
METEOR: 0.317
computing Rouge score...
ROUGE_L: 0.572
computing CIDEr score...
CIDEr: 1.092
Creating temp directory  /tmp/e2e-eval-0z53x6m8
Running MTEval to compute BLEU & NIST...
Use of 'Hyphen' in \p{} or \P{} is deprecated because: Supplanted by Line_Break property values; see www.unicode.org/reports/tr14; at /data/private/E2E/e2e-metrics/mteval/mteval-v13a-sig.pl line 993.
MT evaluation scorer began on 2020 Aug 11 at 10:23:28
command line:  /data/private/E2E/e2e-metrics/mteval/mteval-v13a-sig.pl -r /tmp/e2e-eval-0z53x6m8/mteval_ref.sgm